### Install required external libraries 

In [0]:
%pip install transformers==4.30.2 "unstructured[pdf,docx]==0.10.30" langchain==0.1.5 llama-index==0.9.3 databricks-vectorsearch==0.22 pydantic==1.10.9 mlflow==2.10.1
dbutils.library.restartPython()

  Obtaining dependency information for transformers==4.30.2 from https://files.pythonhosted.org/packages/5b/0b/e45d26ccd28568013523e04f325432ea88a442b4e3020b757cf4361f0120/transformers-4.30.2-py3-none-any.whl.metadata
  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
  Obtaining dependency information for unstructured[docx,pdf]==0.10.30 from https://files.pythonhosted.org/packages/e3/95/28c08bbdd98a9bdbb7f52c07c0e81090ea5f89af5b6cac7c7c6d86ce2a5e/unstructured-0.10.30-py3-none-any.whl.metadata
  Using cached unstructured-0.10.30-py3-none-any.whl.metadata (25 kB)
  Obtaining dependency information for langchain==0.1.5 from https://files.pythonhosted.org/packages/c1/c3/0e59a0c24e0c61b52271445df55302ab2f3dd8489a365721c7ef7ecaba24/langchain-0.1.5-py3-none-any.whl.metadata
  Using cached langchain-0.1.5-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for llama-index==0.9.3 from https://files.pythonhosted.org/packages/e3/fa/94e09b6b35efa90451e060cc1ac1

### Init our resources and catalog

In [0]:
%run ./00-init $reset_all_data=false

###pdf files are available in Volume (or DBFS)

In [0]:
# List our raw PDF docs
volume_folder =  f"/Volumes/{catalog}/{db}/documents_store/pdf_documents"   

display(dbutils.fs.ls(volume_folder))

path,name,size,modificationTime
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso Quarterly Financial Report - December 2023.pdf,Contoso Quarterly Financial Report - December 2023.pdf,137216,1753967249000
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso Quarterly Financial Report - March 2024.pdf,Contoso Quarterly Financial Report - March 2024.pdf,111983,1753967249000
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso Quarterly Market SentimentReport - December 2023.pdf,Contoso Quarterly Market SentimentReport - December 2023.pdf,207724,1753967250000
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso Quarterly Market SentimentReport - March 2024.pdf,Contoso Quarterly Market SentimentReport - March 2024.pdf,203184,1753967250000
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso_improve_customer_churn.pdf,Contoso_improve_customer_churn.pdf,634144,1753967249000


### Transform pdf as text

In [0]:
from unstructured.partition.auto import partition
import re

def extract_doc_text(x : bytes) -> str:
  # Read files and extract the values with unstructured
  sections = partition(file=io.BytesIO(x))
  def clean_section(txt):
    txt = re.sub(r'\n', '', txt)
    return re.sub(r' ?\.', '.', txt)
  # Default split is by section of document, concatenate them all together because we want to split by sentence instead.
  return "\n".join([clean_section(s.text) for s in sections]) 

2025-07-31 17:35:55.443171: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-31 17:35:55.470427: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-31 17:35:55.580390: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-31 17:35:57.763390: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [0]:
%sql
--Note that we need to enable Change Data Feed on the table to create the index
CREATE TABLE IF NOT EXISTS litware_unity_catalog.rag.documents_embedding_openai (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  url STRING,
  content STRING,
  embedding STRING
) TBLPROPERTIES (delta.enableChangeDataFeed = true); 

#### Create the final documents_embedding table containing chunks and embeddings

In [0]:
from pyspark.sql import SparkSession, functions as F
import io
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index import Document, set_global_tokenizer
from transformers import AutoTokenizer

# Reduce the arrow batch size as our PDF can be big in memory
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 10)

# Define the function to read data as chunks
def read_as_chunk(text):
    # set llama2 as tokenizer to match our model size (will stay below BGE 1024 limit)
    set_global_tokenizer(
        AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
    )
    # Sentence splitter from llama_index to split on sentences
    splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)
    txt = extract_doc_text(text)
    nodes = splitter.get_nodes_from_documents([Document(text=txt)])
    return [n.text for n in nodes]

# Register the UDF
read_as_chunk_udf = F.udf(read_as_chunk, F.ArrayType(F.StringType()))

# Read data from the 'pdf_raw' table
df = spark.read.table(f"{catalog}.{db}.documents_raw")

# Use withColumn and the UDF to add a new column
df = df.withColumn("text", F.explode(read_as_chunk_udf(df.content)))

# Show the modified DataFrame
df.show(9)

+--------------------+-------------------+------+--------------------+--------------------+
|                path|   modificationTime|length|             content|                text|
+--------------------+-------------------+------+--------------------+--------------------+
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|Quarterly Market ...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|In summary, Conto...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|Highlight new ser...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|Highly Unsatisfie...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|Highly Unsatisfie...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|It has been nothi...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|I will not be rec...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|I would no

### Computing the chunk embeddings and saving them to our Delta Table


In [0]:
import mlflow.deployments
from pyspark.sql import functions as F
from pyspark.sql.functions import *

def get_embedding(text: str) -> list[float]:
    deploy_client = mlflow.deployments.get_deploy_client("databricks")  # Assuming MLflow is configured

    try:
        response = deploy_client.predict(endpoint="text-embedding-ada-002-Azure-OpenAI", inputs={"input": [text]})  # databricks-bge-large-en
        return response.data[0]['embedding']  # Extract embedding for the single input
    except Exception as e:
        return None  # Handle potential errors

# Read data from the 'pdf_raw' table
#df = spark.read.table(f"{catalog}.{db}.fabcon_pdf_raw")

get_embedding_udf = F.udf(get_embedding, F.StringType())  # Adjusted return type for single string
df = df.withColumn("embedding", get_embedding_udf(F.col("text")))  # Apply UDF to text column

df.show(2)

+--------------------+-------------------+------+--------------------+--------------------+--------------------+
|                path|   modificationTime|length|             content|                text|           embedding|
+--------------------+-------------------+------+--------------------+--------------------+--------------------+
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|Quarterly Market ...|[-0.013238723, -0...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|In summary, Conto...|[-0.014006799, -0...|
+--------------------+-------------------+------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [0]:
from pyspark.sql.functions import concat

df.selectExpr('path as url', 'text as content', 'embedding') \
  .write \
  .mode("append") \
  .format("delta") \
  .saveAsTable(f"{catalog}.{db}.documents_embedding_openai")